# 20. Auditoria: Detecção de Anomalias em Compras (LangGraph + PyOD)

Este notebook demonstra um pipeline automatizado de auditoria de compras usando:
*   **PyOD (Isolation Forest):** Para detecção estatística de anomalias (outliers) nos valores das compras.
*   **LangGraph:** Para orquestrar o fluxo de trabalho (Carregar Dados -> Detectar Anomalias -> Analisar com IA).
*   **LangChain + Google Gemini:** Para interpretar os resultados e gerar um relatório de auditoria em linguagem natural.

**Cenário:** Temos um dataset de compras corporativas e queremos identificar transações suspeitas (valores muito altos ou fora do padrão).

In [ ]:
### INJECTION START ###
import os
from dotenv import load_dotenv
# Carregar variáveis de ambiente locais se existirem (para rodar localmente)
load_dotenv()
### INJECTION END ###

# Instalação das bibliotecas necessárias (Executar no Colab)
!pip install -qU langchain langchain-google-genai langgraph pyod pandas

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Configuração da API Key do Google
try:
    from google.colab import userdata
    if not os.getenv("GOOGLE_API_KEY"):
        os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except ImportError:
    pass # Assume que já está no ambiente se rodar local

## 1. Definindo o Estado do Grafo

O estado (`AuditState`) armazena os dados que fluem entre os nós do grafo.

In [ ]:
from typing import TypedDict, List, Dict, Any

class AuditState(TypedDict):
    raw_data: pd.DataFrame          # Dataset completo
    processed_data: pd.DataFrame    # Dataset com scores de anomalia
    anomalies: pd.DataFrame         # Apenas os registros anômalos
    analysis_report: str            # Relatório final gerado pela IA

## 2. Definindo os Nós (Nodes)

### Nó 1: Gerador de Dados (Mock)
Simula um banco de dados de compras com algumas anomalias inseridas propositalmente.

In [ ]:
def load_data_node(state: AuditState) -> Dict[str, Any]:
    print("--- [PASSO 1] Gerando Dados de Compras ---")
    
    # Gerar 100 transações normais
    data = []
    for i in range(100):
        data.append({
            "id": i,
            "data": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
            "fornecedor": f"Fornecedor {random.choice(['A', 'B', 'C', 'D'])}",
            "departamento": random.choice(['TI', 'RH', 'Marketing', 'Operações']),
            "valor": round(random.uniform(100, 5000), 2)  # Valores normais entre 100 e 5000
        })
    
    # Inserir 5 ANOMALIAS (Outliers)
    anomalies_data = [
        {"id": 101, "data": "2024-02-01", "fornecedor": "Fornecedor X", "departamento": "TI", "valor": 55000.00}, # Valor muito alto
        {"id": 102, "data": "2024-02-02", "fornecedor": "Fornecedor Y", "departamento": "RH", "valor": 42000.50},
        {"id": 103, "data": "2024-02-03", "fornecedor": "Fornecedor Z", "departamento": "Marketing", "valor": 98000.00},
        {"id": 104, "data": "2024-02-04", "fornecedor": "Fornecedor A", "departamento": "TI", "valor": 5.00},     # Valor muito baixo (suspeito?)
        {"id": 105, "data": "2024-02-05", "fornecedor": "Fornecedor X", "departamento": "Operações", "valor": 62000.00}
    ]
    data.extend(anomalies_data)
    
    df = pd.DataFrame(data)
    return {"raw_data": df}

### Nó 2: Detector de Anomalias (PyOD)
Usa o algoritmo `IsolationForest` para identificar outliers baseados na coluna `valor`.

In [ ]:
from pyod.models.iforest import IForest

def detect_anomalies_node(state: AuditState) -> Dict[str, Any]:
    print("--- [PASSO 2] Detectando Anomalias com Isolation Forest ---")
    df = state["raw_data"].copy()
    
    # Prepara os dados para o PyOD (precisa ser 2D array)
    X = df[['valor']].values
    
    # Treina o Isolation Forest
    # contamination=0.05 significa que esperamos que aprox. 5% dos dados sejam anomalias
    clf = IForest(contamination=0.05, random_state=42)
    clf.fit(X)
    
    # Previsão: 1 = outlier, 0 = inlier
    # Nota: PyOD retorna 0 para inlier e 1 para outlier nos métodos predict
    df['is_anomaly'] = clf.predict(X)
    
    # Score de anomalia (quanto maior, mais anômalo)
    df['anomaly_score'] = clf.decision_function(X)
    
    # Filtra apenas as anomalias para passar para o próximo passo
    anomalies_df = df[df['is_anomaly'] == 1].sort_values(by='valor', ascending=False)
    
    print(f"Anomalias detectadas: {len(anomalies_df)}")
    
    return {"processed_data": df, "anomalies": anomalies_df}

### Nó 3: Analista IA (LangChain)
Recebe os dados das anomalias e gera um parecer técnico explicando o risco.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

def analyze_node(state: AuditState) -> Dict[str, Any]:
    print("--- [PASSO 3] Analisando Riscos com IA ---")
    anomalies = state["anomalies"]
    
    if anomalies.empty:
        return {"analysis_report": "✅ Nenhuma anomalia significativa foi detectada nesta rodada de auditoria."}
    
    # Converte os dados anômalos para string (Markdown/CSV) para o prompt
    data_text = anomalies[['id', 'data', 'fornecedor', 'departamento', 'valor', 'anomaly_score']].to_markdown(index=False)
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    
    prompt = ChatPromptTemplate.from_template(
        """Você é um Auditor Sênior especializado em detecção de fraudes corporativas.
        
        O sistema de Machine Learning (Isolation Forest) detectou as seguintes transações anômalas:
        
        {data}
        
        Sua tarefa:
        1. Analise os valores e fornecedores.
        2. Explique POR QUE essas transações parecem suspeitas (ex: valor muito acima da média, outliers estatísticos).
        3. Categorize o risco (Alto/Médio/Baixo).
        4. Recomende uma ação para o auditor humano (ex: "Solicitar nota fiscal", "Verificar contrato").
        
        Formate a saída como um relatório Markdown profissional.
        """
    )
    
    chain = prompt | llm
    response = chain.invoke({"data": data_text})
    
    return {"analysis_report": response.content}

## 3. Construindo o Grafo (Graph)

Conectamos os nós em um fluxo sequencial.

In [ ]:
from langgraph.graph import StateGraph, END

# Inicializa o grafo com o tipo de estado
workflow = StateGraph(AuditState)

# Adiciona os nós
workflow.add_node("load_data", load_data_node)
workflow.add_node("detect_anomalies", detect_anomalies_node)
workflow.add_node("analyze_report", analyze_node)

# Define as arestas (fluxo)
workflow.set_entry_point("load_data")
workflow.add_edge("load_data", "detect_anomalies")
workflow.add_edge("detect_anomalies", "analyze_report")
workflow.add_edge("analyze_report", END)

# Compila o grafo
app = workflow.compile()

## 4. Executando a Auditoria

Rodamos o grafo e exibimos o relatório final.

In [ ]:
# Visualizando o fluxo (opcional, requer graphviz)
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except:
    pass

# Executa o workflow
result = app.invoke({})

# Exibe o Relatório Final
print("\n" + "="*40)
print("    RELATÓRIO DE AUDITORIA IA")
print("="*40 + "\n")
from IPython.display import Markdown
display(Markdown(result["analysis_report"]))

### Visualizando os Dados Brutos (Opcional)

In [ ]:
# Mostra as anomalias encontradas no dataframe
result["anomalies"]